### TF - motif bias matrix

1. downloading gene - motif name mapping in here:
   https://resources.aertslab.org/cistarget/motif_collections/v10nr_clust_public/snapshots/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl
2. then load the motif -> gene enrichment mapping in here:
   https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc_v10_clust/gene_based/the
   hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.scores.feather
3. then we replace the motif name with gene names
4. then we take the max enrichment if same gene-motif
5. then renormalize this per gene and save it as our new attention bias

---

### PPI bias matrix

1. we get the human PPI matrix from stringdb here:
   https://string-db.org/cgi/download?sessionId=bwVBywlOX6i8&species_text=Homo+sapiens
2. we get the same one for other species (e.g. mus musculus):
   https://stringdb-downloads.org/download/protein.links.v12.0/10090.protein.links.v12.0.txt.gz
3. we open them and use ensembl's biomart to map ENSP to ENSG. we merge all
   similar pairs
4. we map it to a grn with associated scores.
5. we use them as our new PPI bias matrix


In [1]:
import pandas as pd
from scdataloader.utils import load_genes
from scdataloader.utils import getBiomartTable
import numpy as np
from tqdm import tqdm
import torch
from scipy import sparse as sp

%load_ext autoreload
%autoreload 2


→ connected lamindb: jkobject/scprint2


In [2]:
genedf = load_genes()

In [3]:
LOC = "../../data/main/"  # "./data/main/"

In [4]:
da = pd.read_feather(LOC + "main_scenic+_database.feather")
da = da.set_index("motifs")


tbl = pd.read_csv(
    LOC + "motifs-v10-nr.hgnc-m0.00001-o0.0.tbl",
    sep="\t",
    header=None,
    names=[
        "motif_id",
        "motif_name",
        "motif_description",
        "source_name",
        "source_version",
        "gene_name",
        "motif_similarity_qvalue",
        "similar_motif_id",
        "similar_motif_description",
        "orthologous_identity",
        "orthologous_gene_name",
        "orthologous_species",
        "description",
    ],
)
tbl = tbl.set_index("motif_id")
tbl = tbl[tbl.gene_name.isin(genedf.symbol)]

/local/scratch/tmp/ipykernel_433531/1070051262.py:5: DtypeWarning: Columns (4,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  tbl = pd.read_csv(


In [5]:
rn = {}
toadd = {}
for i in tqdm(da.index):
    try:
        res = tbl.loc[i, "gene_name"]
    except KeyError:
        da = da.drop(i)
    if type(res) != str:
        for v in res.values[1:]:
            toadd.update({v: da.loc[i].values})
        res = res.values[0]
    rn.update({i: res})

100%|██████████| 5876/5876 [00:46<00:00, 125.53it/s]


In [6]:
da = da.rename(index=rn)
da = pd.concat([da, pd.DataFrame(toadd, index=da.columns).T])
da = da.loc[~da.index.duplicated(keep="first")]

In [38]:
da.to_parquet(LOC + "main_scenic+_2.parquet")

In [39]:
da = pd.read_parquet(LOC + "main_scenic+_2.parquet")

In [7]:
biomart = getBiomartTable(
    attributes=["ensembl_gene_id", "hgnc_symbol"], bypass_attributes=True
)

downloading gene names from biomart
['ensembl_gene_id', 'hgnc_symbol']


In [8]:
mp = {}
for v, k in biomart.iloc[:, :2].values:
    if v is np.nan or k is np.nan:
        continue
    if k in mp:
        if v not in mp[k]:
            mp[k] += [v]
    else:
        mp[k] = [v]

In [9]:
MAX = 5
MIN = 0

da = (
    MIN
    + (da - da.min(1).values[:, None])
    * (MAX - MIN)
    / (da.max(1) - da.min(1)).values[:, None]
)
da.shape

(1564, 27090)

In [16]:
new_index = []
new_data = []

# Remap indices
for idx in da.index:
    if idx in mp:
        for mapped_idx in mp[idx]:
            new_index.append(mapped_idx)
            new_data.append(da.loc[idx].values)
new_data = np.array(new_data)
new_data.shape

(1692, 27090)

In [18]:
da = pd.DataFrame(new_data, index=new_index, columns=da.columns)

In [23]:
new_columns = []
new_data = []
da = da.T

new_index = []
new_data = []

# Remap indices
for idx in da.index:
    if idx in mp:
        for mapped_idx in mp[idx]:
            new_index.append(mapped_idx)
            new_data.append(da.loc[idx].values)
new_data = np.array(new_data)
new_data.shape

(26639, 1692)

In [25]:
da = pd.DataFrame(new_data.T, index=da.columns, columns=new_index)

In [ ]:
da = da[~da.index.duplicated(keep="first")]
da = da.T
da = da[~da.index.duplicated(keep="first")]
da = da.T

## protein


In [ ]:
string = pd.read_csv(LOC + "9606.protein.links.v12.0.txt.gz", sep=" ")

In [ ]:
string.protein1 = string.protein1.str.split(".").str[1]
string.protein2 = string.protein2.str.split(".").str[1]

In [ ]:
rel = {}
for a, b in string.iloc[:, :2].values:
    a, b = rn.get(a, ""), rn.get(b, "")
    if a and b:
        if a in rel:
            rel[a].add(b)
        else:
            rel[a] = set([b])
        if b in rel:
            rel[b].add(a)
        else:
            rel[b] = set([a])
len(rel)

19193

In [ ]:
res = pd.DataFrame(
    data=np.zeros((len(rel), len(rel))),
    columns=list(rel.keys()),
    index=list(rel.keys()),
)

In [ ]:
res.drop(columns=[""], inplace=True)
res.drop(index=[""], inplace=True)

In [ ]:
for i, j in tqdm.tqdm(rel.items()):
    res.loc[i, list(j)] = 1

19193it [00:20, 917.76it/s] 


In [ ]:
res.mean().mean()

0.03608647264434039

In [ ]:
res.to_parquet(LOC + "stringdb_bias.parquet")

In [ ]:
res = pd.read_parquet(LOC + "stringdb_bias.parquet")

In [36]:
res = res + res.T

In [35]:
res.sum(1)

ENSG00000075292    1221.0
ENSG00000172531    3264.0
ENSG00000180745     466.0
ENSG00000156886     287.0
ENSG00000185069     671.0
                    ...  
ENSG00000178093    1031.0
ENSG00000203950       0.0
ENSG00000126953     852.0
ENSG00000185985     719.0
ENSG00000180210    1276.0
Length: 19193, dtype: float64

In [41]:
len(set(da.columns))

23165

In [ ]:
set(da.columns)

,ENSG00000121410,ENSG00000268895,ENSG00000148584,ENSG00000175899,ENSG00000245105,ENSG00000166535,ENSG00000256069,ENSG00000184389,ENSG00000128274,ENSG00000118017,ENSG00000291836,ENSG00000081760,ENSG00000250420,ENSG00000114771,ENSG00000197953,ENSG00000242908,ENSG00000188984,ENSG00000204518,ENSG00000240602,ENSG00000109576,ENSG00000103591,ENSG00000115977,ENSG00000087884,ENSG00000127837,ENSG00000129673,ENSG00000131043,ENSG00000205002,ENSG00000124608,ENSG00000266967,ENSG00000157426,ENSG00000149313,ENSG00000008311,ENSG00000215458,ENSG00000276072,ENSG00000181409,ENSG00000281376,ENSG00000183044,ENSG00000165029,ENSG00000154263,ENSG00000251595,ENSG00000144452,ENSG00000179869,ENSG00000238098,ENSG00000107331,ENSG00000167972,ENSG00000198691,ENSG00000154265,ENSG00000154262,ENSG00000064687,ENSG00000141338,ENSG00000154258,ENSG00000231749,ENSG00000085563,ENSG00000135776,ENSG00000276582,ENSG00000005471,ENSG00000004846,ENSG00000115657,ENSG00000131269,ENSG00000197150,ENSG00000150967,ENSG00000278183,ENSG00000124574,ENSG00000121270,ENSG00000140798,ENSG00000243064,ENSG00000023839,ENSG00000108846,ENSG00000125257,ENSG00000114770,ENSG00000223882,ENSG00000275331,ENSG00000291057,ENSG00000273808,ENSG00000006071,ENSG00000069431,ENSG00000101986,ENSG00000173208,ENSG00000117528,ENSG00000119688,ENSG00000164163,ENSG00000204574,ENSG00000033050,ENSG00000161204,ENSG00000160179,ENSG00000118777,ENSG00000172350,ENSG00000138075,ENSG00000143921,ENSG00000143994,ENSG00000144827,ENSG00000106077,ENSG00000100997,ENSG00000131969,ENSG00000139826,ENSG00000248487,ENSG00000114786,ENSG00000114779,ENSG00000168792,ENSG00000204427,...,ENSG00000124203,ENSG00000267343,ENSG00000127903,ENSG00000196267,ENSG00000152475,ENSG00000022976,ENSG00000198040,ENSG00000197608,ENSG00000176723,ENSG00000223547,ENSG00000213799,ENSG00000196605,ENSG00000105750,ENSG00000267041,ENSG00000281626,ENSG00000236609,ENSG00000197385,ENSG00000106479,ENSG00000261221,ENSG00000290382,ENSG00000257446,ENSG00000234284,ENSG00000221923,ENSG00000228623,ENSG00000213793,ENSG00000159904,ENSG00000214029,ENSG00000213988,ENSG00000167232,ENSG00000146757,ENSG00000184635,ENSG00000197360,ENSG00000213973,ENSG00000124201,ENSG00000106400,ENSG00000174276,ENSG00000278574,ENSG00000117174,ENSG00000186187,ENSG00000180233,ENSG00000237004,ENSG00000225264,ENSG00000183579,ENSG00000177993,ENSG00000105428,ENSG00000149506,ENSG00000284588,ENSG00000188372,ENSG00000116996,ENSG00000042813,ENSG00000186075,ENSG00000170044,ENSG00000109917,ENSG00000019995,ENSG00000132485,ENSG00000235079,ENSG00000121988,ENSG00000169249,ENSG00000152467,ENSG00000130182,ENSG00000158691,ENSG00000219891,ENSG00000196812,ENSG00000269293,ENSG00000121413,ENSG00000291625,ENSG00000121903,ENSG00000166529,ENSG00000182318,ENSG00000187987,ENSG00000197037,ENSG00000197062,ENSG00000140265,ENSG00000186814,ENSG00000235109,ENSG00000140987,ENSG00000180532,ENSG00000131848,ENSG00000197213,ENSG00000137185,ENSG00000168612,ENSG00000163012,ENSG00000132801,ENSG00000288360,ENSG00000162415,ENSG00000130449,ENSG00000214941,ENSG00000214655,ENSG00000272589,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000285443,ENSG00000074755,ENSG00000036549
ENSG00000168214,0.060400,0.060800,0.197400,0.128600,0.082600,0.149600,0.095200,0.150200,0.125000,0.119600,0.147600,0.181400,0.126000,0.090200,0.083600,0.177000,0.156400,0.086200,0.097400,0.100200,0.093000,0.087600,0.130600,0.135800,0.076400,0.150000,0.230000,0.125400,0.141200,0.079000,0.069000,0.154000,0.117000,0.121800,0.162000,0.143400,0.109000,0.164800,0.094600,0.081600,0.146800,0.116800,0.063400,0.102400,0.131800,0.145000,0.202000,0.133000,0.153200,0.089200,0.116800,0.094800,0.191400,0.140200,0.082000,0.076000,0.175000,0.172800,0.128400,0.089000,0.136200,0.130400,0.110600,0.151200,0.112400,0.146600,0.079400,0.138400,0.097200,0.086600,0.111200,0.081800,0.074200,0.067000,0.070000,0.099800,0.187200,0.142800,0.078800,0.116400,0.139400,0.086400,0.136400,0.067200,0.178400,0.187400,0.254

In [43]:
# # Align the indices and columns of res with da, and vice versa, filling missing values with 0
all_indices = sorted(set(da.index).union(set(res.index)))
#
da_aligned = da.reindex(index=all_indices, columns=all_indices, fill_value=0)
res_aligned = res.reindex(index=all_indices, columns=all_indices, fill_value=0)
#
# # Add res to da
da = da_aligned + res_aligned

In [44]:
da.shape

(19269, 19269)

In [45]:
biomart = getBiomartTable(
    attributes=["mmusculus_homolog_ensembl_gene", "ensembl_gene_id"],
    bypass_attributes=True,
)
biomart

downloading gene names from biomart
['mmusculus_homolog_ensembl_gene', 'ensembl_gene_id']


,mmusculus_homolog_ensembl_gene,ensembl_gene_id
0,NaN,ENSG00000210049
1,NaN,ENSG00000211459
2,NaN,ENSG00000210077
3,NaN,ENSG00000210082
4,NaN,ENSG00000209082
...,...,...
77642,NaN,ENSG00000232679
77643,ENSMUSG00000088001,ENSG00000200033
77644,NaN,ENSG00000228437
77645,NaN,ENSG00000229463


In [46]:
biomart[~biomart.mmusculus_homolog_ensembl_gene.isna()]

,mmusculus_homolog_ensembl_gene,ensembl_gene_id
5,ENSMUSG00000064341,ENSG00000198888
9,ENSMUSG00000064345,ENSG00000198763
15,ENSMUSG00000064351,ENSG00000198804
18,ENSMUSG00000064354,ENSG00000198712
20,ENSMUSG00000064356,ENSG00000228253
...,...,...
77627,ENSMUSG00000032913,ENSG00000198799
77639,ENSMUSG00000102439,ENSG00000143631
77640,ENSMUSG00000049133,ENSG00000143520
77641,ENSMUSG00000039384,ENSG00000143507


In [ ]:
mp = {}
for v, k in biomart.values:
    if v is np.nan or k is np.nan:
        continue
    if k in mp:
        if v not in mp[k]:
            mp[k] += [v]
    else:
        mp[k] = [v]

In [53]:
da2 = da.copy()
da2 = da2.loc[da2.index.isin(mp.keys()), da2.columns.isin(mp.keys())]
da2.index = [mp[i][0] for i in da2.index]
da2.columns = [mp[i][0] for i in da2.columns]

In [66]:
da2 = da2[~da2.index.duplicated(keep="first")]
da2 = da2.T
da2 = da2[~da2.index.duplicated(keep="first")]
da2 = da2.T

In [71]:
sparse

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,ENSG00000001497,ENSG00000001561,ENSG00000001617,ENSG00000001626,ENSG00000001629,ENSG00000001630,ENSG00000001631,ENSG00000002016,ENSG00000002330,ENSG00000002549,ENSG00000002586,ENSG00000002587,ENSG00000002726,ENSG00000002745,ENSG00000002746,ENSG00000002822,ENSG00000002834,ENSG00000002919,ENSG00000002933,ENSG00000003056,ENSG00000003096,ENSG00000003137,ENSG00000003147,ENSG00000003249,ENSG00000003393,ENSG00000003400,ENSG00000003402,ENSG00000003436,ENSG00000003509,ENSG00000003756,ENSG00000003987,ENSG00000003989,ENSG00000004059,ENSG00000004139,ENSG00000004142,ENSG00000004399,ENSG00000004455,ENSG00000004468,ENSG00000004478,ENSG00000004487,ENSG00000004534,ENSG00000004660,ENSG00000004700,ENSG00000004766,ENSG00000004776,ENSG00000004777,ENSG00000004779,ENSG00000004799,ENSG00000004809,ENSG00000004838,ENSG00000004846,ENSG00000004848,ENSG00000004864,ENSG00000004866,ENSG00000004897,ENSG00000004939,ENSG00000004961,ENSG00000004975,ENSG00000005001,ENSG00000005007,ENSG00000005020,ENSG00000005022,ENSG00000005059,ENSG00000005073,ENSG00000005075,ENSG00000005100,ENSG00000005102,ENSG00000005108,ENSG00000005156,ENSG00000005175,ENSG00000005187,ENSG00000005189,ENSG00000005194,ENSG00000005206,ENSG00000005238,ENSG00000005243,ENSG00000005249,ENSG00000005302,ENSG00000005339,ENSG00000005379,ENSG00000005381,ENSG00000005421,ENSG00000005436,ENSG00000005448,ENSG00000005469,ENSG00000005471,ENSG00000005483,ENSG00000005486,...,ENSMUSG00000045140,ENSMUSG00000047246,ENSMUSG00000018537,ENSMUSG00000038453,ENSMUSG00000070337,ENSMUSG00000069662,ENSMUSG00000048827,ENSMUSG00000022579,ENSMUSG00000108348,ENSMUSG00000022055,ENSMUSG00000069744,ENSMUSG00000038541,ENSMUSG00000073913,ENSMUSG00000078695,ENSMUSG00000010362,ENSMUSG00000020677,ENSMUSG00000062564,ENSMUSG00000060397,ENSMUSG00000044933,ENSMUSG00000096549,ENSMUSG00000020527,ENSMUSG00000020530,ENSMUSG00000071478,ENSMUSG00000051452,ENSMUSG00000034449,ENSMUSG00000020532,ENSMUSG00000032292,ENSMUSG00000058385,ENSMUSG00000071653,ENSMUSG00000018405,ENSMUSG00000038199,ENSMUSG00000096010,ENSMUSG00000067771,ENSMUSG00000018882,ENSMUSG00000045792,ENSMUSG00000066672,ENSMUSG00000064333,ENSMUSG00000062142,ENSMUSG00000108948,ENSMUSG00000075113,ENSMUSG00000050577,ENSMUSG00000066747,ENSMUSG00000075377,ENSMUSG00000050505,ENSMUSG00000075186,ENSMUSG00000030680,ENSMUSG00000078320,ENSMUSG00000000562,ENSMUSG00000108398,ENSMUSG00000034185,ENSMUSG00000107877,ENSMUSG00000070902,ENSMUSG00000031452,ENSMUSG00000087512,ENSMUSG00000073234,ENSMUSG00000090840,ENSMUSG00000073598,ENSMUSG00000079304,ENSMUSG00000105734,ENSMUSG00000096537,ENSMUSG00000068167,ENSMUSG00000109737,ENSMUSG00000107252,ENSMUSG00000079497,ENSMUSG00000110221,ENSMUSG00000114865,ENSMUSG00000098590,ENSMUSG00000048752,ENSMUSG00000095975,ENSMUSG00000049761,ENSMUSG00000043795,ENSMUSG00000091780,ENSMUSG00000030030,ENSMUSG00000114245,ENSMUSG00000110170,ENSMUSG00000049098,ENSMUSG00000114004,ENSMUSG00000053297,ENSMUSG00000037689,ENSMUSG00000109305,ENSMUSG00000115219,ENSMUSG00000039233,ENSMUSG00000115302,ENSMUSG00000095026,ENSMUSG00000078630,ENSMUSG00000027676,ENSMUSG00000041857,ENSMUSG00000115798,ENSMUSG00000116563,ENSMUSG00000096530,ENSMUSG00000103523,ENSMUSG00000117809,ENSMUSG00000091614,ENSMUSG00000117748,ENSMUSG00000018451,ENSMUSG00000092349,ENSMUSG00000047150,ENSMUSG00000064220,ENSMUSG00000057156,ENSMUSG00000006818
ENSG00000000003,0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000000005,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [70]:
sparse.index.nunique()

35989

In [68]:
sparse = pd.merge(
    da,
    da2,
    left_index=True,
    right_index=True,
    how="outer",
)
sparse.fillna(0, inplace=True)

In [ ]:
sparse = sparse.astype(pd.SparseDtype("float", 0))

In [ ]:

# Convert to scipy sparse
sparse_matrix = sp.csr_matrix(sparse.values)

# Save the matrix and indices separately
sp.save_npz(LOC + "bias_sparse.npz", sparse_matrix)
pd.Series(sparse.index).to_csv(LOC + "bias_sparse_index.csv")
pd.Series(sparse.columns).to_csv(LOC + "bias_sparse_columns.csv")

AttributeError: 'Series' object has no attribute 'to_parquet'

In [80]:
del sparse, sparse_matrix, da, da2

In [ ]:
sparse_matrix = sp.load_npz(LOC + "bias_sparse.npz")
index = pd.read_csv(LOC + "bias_sparse_index.csv").iloc[:, 1].values

In [9]:
del sparse

In [7]:
bias = ad.AnnData(X=np.zeros((1, len(index))), var=index)

In [10]:
bias.varp['GRN'] = sparse_matrix

In [11]:
bias = GRNAnnData(bias)

In [23]:
bias.var.columns = ["index"]
bias.var = bias.var.set_index("index")

In [24]:
bias.write(LOC+'bias.h5ad')

In [26]:
bias = read_h5ad(LOC+'bias.h5ad')

In [ ]:
bias[bias.var.index.isin()]

In [25]:
import anndata as ad
from grnndata import GRNAnnData, read_h5ad
from bengrn import BenGRN